In [47]:
import pandas as pd
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'

df = pd.read_excel(url)



def delete_columns(df):
    df_dropped_multiple = df.drop(['Year', 'Type','Location','Name','Sex','Age','Injury','Unnamed: 11','Time','Species ','Source','pdf','href formula','href','Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22',],axis=1)
    return df_dropped_multiple
print(delete_columns(df))
df= delete_columns(df)

def clean_data(df):
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # Remove duplicates
    df.drop_duplicates(inplace=True)
    
    #Delete null values
    df.dropna()
    df=df.dropna()
    return df
print(clean_data(df))


def remove_prefix(date):
    # Make sure date is a string
    if isinstance(date, str) and date.startswith("Reported "):
        # If the date starts with "Reported", we return what comes after that
        return date[len("Reported "):]
    # Return the original value if its not a string or doesn't start with the prefix
    return date

def fix_format_date(date):
    # Make sure date is a string before trying to convert it
    if isinstance(date, str):
        try:
            # Convert the value to datetime format with the format dd-mmm-yyyy
            correct_date_format = pd.to_datetime(date, format='%d-%b-%Y', errors='raise')
            # Return the date in dd-mm-yyyy format
            return correct_date_format.strftime('%d-%m-%Y')
        except ValueError:
            return None
    # If date is not a string we return None
    return None

df['date'] = df['date'].apply(remove_prefix)
df['date'] = df['date'].apply(fix_format_date)

def filter_date(df):

    df =df[df.date >= pd.to_datetime('2020-01-01') ]

    return df
print(df)



             Date    Country              State  Activity
0     15 Mar 2024  AUSTRALIA         Queensland  Swimming
1     04 Mar 2024        USA             Hawaii   Surfing
2     02 Mar-2024        USA             Hawaii  Swimming
3     25 Feb-2024  AUSTRALIA  Western Australia       NaN
4     14 Feb-2024      INDIA        Maharashtra   Fishing
...           ...        ...                ...       ...
6964          NaN        NaN                NaN       NaN
6965          NaN        NaN                NaN       NaN
6966          NaN        NaN                NaN       NaN
6967          NaN        NaN                NaN       NaN
6968          NaN        NaN                NaN       NaN

[6969 rows x 4 columns]
             date             country              state      activity
0     15 Mar 2024           AUSTRALIA         Queensland      Swimming
1     04 Mar 2024                 USA             Hawaii       Surfing
2     02 Mar-2024                 USA             Hawaii      Swim

In [52]:
df.date= pd.to_datetime(df.date,  errors='coerce')

In [53]:
df[df.date >= pd.to_datetime('2020-01-01') ]


,date,country,state,activity
5,2024-04-02,TRINIDAD,NaN,Spearfishing
8,2024-09-01,AUSTRALIA,South Australia,Surfing
9,2024-05-01,SOUTH AFRICA,Eastern Cape Province,Fishing
43,2023-06-09,Coral Sea,NaN,On a round-the-world expedition
74,2023-07-06,BAHAMAS,Freeport,Scuba diving
...,...,...,...,...
421,2020-12-01,THAILAND,Phang Nga Province,Swimming
422,2020-07-01,AUSTRALIA,Queensland,Swimming
423,2020-05-01,AUSTRALIA,Western Australia,Scuba diving
424,2020-03-01,NEW CALEDONIA,NaN,NaN


In [11]:
import pandas as pd
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'

df = pd.read_excel(url)

def clean_data(df):
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # Remove duplicates
    df.drop_duplicates(inplace=True)
    
    #Delete null values
    df_nullValues = df.dropna()
    return df
print(clean_data(df))

             date    year        type             country              state  \
0     15 Mar 2024  2024.0  Unprovoked           AUSTRALIA         Queensland   
1     04 Mar 2024  2024.0  Unprovoked                 USA             Hawaii   
2     02 Mar-2024  2024.0  Unprovoked                 USA             Hawaii   
3     25 Feb-2024  2024.0  Unprovoked           AUSTRALIA  Western Australia   
4     14 Feb-2024  2024.0  Unprovoked               INDIA        Maharashtra   
...           ...     ...         ...                 ...                ...   
6940  Before 1903     0.0  Unprovoked           AUSTRALIA  Western Australia   
6941    1900-1905     0.0  Unprovoked                 USA     North Carolina   
6942    1883-1889     0.0  Unprovoked              PANAMA                NaN   
6943    1845-1853     0.0  Unprovoked  CEYLON (SRI LANKA)   Eastern Province   
6944          NaN     NaN         NaN                 NaN                NaN   

                                 locati